In [ ]:
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'

#### Create a connection to database file

In [65]:
# Create a connection to a new or existing database file
connection = sqlite3.connect("tornado_tracks.db")

print("Connected:", connection)



Connected: <sqlite3.Connection object at 0x000001774BE095D0>


#### Load tables from CSV files

In [66]:
weather_detail = pd.read_csv("../Datasets/Clean Files/weather_detail_cleaned.csv")
weather_detail.to_sql("weather_detail", connection, if_exists="replace", index=False)


17723

In [67]:
tornado_tracks = pd.read_csv("../Datasets/Clean Files/tornado_tracks_combined_cleaned.csv")
tornado_tracks.to_sql("tornado_tracks", connection, if_exists="replace", index=False)

530

#### Testing to make sure tables loaded

In [68]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)

,name
0,weather_detail
1,tornado_tracks


In [ ]:
pd.read_sql("SELECT * FROM tornado_tracks ;", connection)

In [ ]:

Join = """
SELECT *
FROM tornado_tracks t
inner join weather_detail w
On t.Zip = w.Zip 
AND t.DATE = w.DATE

"""

df = pd.read_sql(Join, connection)
df

#### Tornado Count by Zipcode

In [73]:
Query = """
SELECT Zip, COUNT(*) AS tornado_count
FROM tornado_tracks
GROUP BY Zip
ORDER BY tornado_count DESC
LIMIT 10;
"""
df = pd.read_sql(Query, connection)
df

,Zip,tornado_count
0,40165,38
1,40031,30
2,40207,26
3,40299,25
4,40272,23
5,40026,21
6,40245,20
7,40216,19
8,40059,19
9,40213,17


#### Yearly Tornado Count

In [75]:
Query = """
SELECT strftime('%Y', Date) AS year, COUNT(*) AS tornado_count
FROM tornado_tracks
GROUP BY year
ORDER BY year;
"""
df = pd.read_sql(Query, connection)
df

,year,tornado_count
0,2015,90
1,2016,64
2,2017,31
3,2018,73
4,2019,24
5,2020,21
6,2021,10
7,2022,31
8,2023,71
9,2024,44


#### Yearly Avg Temp

In [82]:
Query = """
SELECT strftime('%Y', Date) as Year, AVG(TMAX) AS AvgTemp
FROM weather_detail
GROUP BY Year
ORDER BY Year;
"""
df = pd.read_sql(Query, connection)
df

,Year,AvgTemp
0,2015,67.707099
1,2016,69.482529
2,2017,69.438674
3,2018,67.747638
4,2019,68.606920
5,2020,68.252273
6,2021,67.743396
7,2022,68.000685
8,2023,69.260810
9,2024,69.981520


#### Close Connection

In [ ]:
# Always close your connection when you're done
connection.close()